In [212]:
using NLsolve

In [213]:
γ=1.4;

Prandtl Meyer Expansion

In [214]:
# nu function
function nu_from_mach(M)
    return sqrt((γ + 1) / (γ - 1)) * atand(sqrt(((γ - 1) / (γ + 1)) * (M^2 - 1))) - atand(sqrt(M^2 - 1))
end;

In [215]:
# Function to compute Mach number from nu
function mach_from_nu(nu_val)
    result = nlsolve(M -> nu_val - nu_from_mach(M[1]), [3.0])
    return result.zero[1]
end;

In [216]:
# Isentropic Pressure ratio function
function ratio_P0_P(M)
    return (1 + ((γ - 1) / 2) * M^2)^(γ / (γ - 1))
end;

In [217]:
# Initial values
M1 = 2.6
θ1 = 5;

In [218]:

ν1 = nu_from_mach(M1)

41.41470777941413

In [219]:

# Compute ν2
ν2 = ν1 + θ1

46.41470777941413

In [220]:
M2 = mach_from_nu(ν2)

2.8322924890653214

In [221]:
P2_P1 = ratio_P0_P(M1) / ratio_P0_P(M2)

0.6999509836036835

Oblique Shock

In [222]:
# theta function
function theta_func(β, M)
    return atand(2 / tand(β) * ((M^2 * sind(β)^2 - 1) / (M^2 * (γ + cosd(2 * β)) + 2)))
end;

In [223]:
# Solve for beta
function find_beta(θ, M)
    result = nlsolve(β -> θ - theta_func(β[1], M), [20.0])
    return result.zero[1]
end;

In [224]:
# Normal Shock Mach Relation
function NS(M)
    return sqrt((1 + ((γ - 1) / 2) * M^2) / (γ * M^2 - (γ - 1) / 2))
end;

In [225]:
# Normal Shock Pressure ratio
function P_ratio(M)
    return 1 + (2 * γ / (γ + 1)) * (M^2 - 1)
end;

In [226]:
β2 = find_beta(θ1, M1)

26.41482914184023

In [227]:
M_N1 = M1 * sind(β2)
M_N3 = NS(M_N1)
M3 = M_N3 / sind(β2 - θ1)
P3_P1 = P_ratio(M_N1)

1.3941570248459896

In [228]:
Cl = (2 / (γ * M1^2)) * (P3_P1 - P2_P1) * cosd(θ1)

0.14614631818868237

In [229]:
Cd = (2 / (γ * M1^2)) * (P3_P1 - P2_P1) * sind(θ1)

0.01278614605756226

In [230]:
function find_phi(φ)
    # Convert φ to a scalar since it's an array input from nlsolve
    φ_value = φ[1]

    # Region 42
    β42 = find_beta(θ1 + φ_value, M2)
    M_N42 = NS(M2 * sind(β42) )  # Compute normal Mach numbers directly
    P42_P2 = P_ratio(M2 * sind(β42))
    M4 = M_N42 / sind(β42 - (θ1 + φ_value))

    # Region 43
    ν43 = nu_from_mach(M3) + θ1 + φ_value
    M43 = mach_from_nu(ν43)
    P43_P3 = ratio_P0_P(M3) / ratio_P0_P(M43)

    return P42_P2 * P2_P1 - P43_P3 * P3_P1
end;

In [231]:
# Initial guess for φ
φ_initial_guess = [0.5]

# Solve for φ using nlsolve
result_phi = nlsolve(find_phi, φ_initial_guess)
φ_solution = result_phi.zero[1]

0.0026771436446359873